In [ ]:
import json
import requests
from pathlib import Path

from string import Formatter
from pathlib import Path


from openai import OpenAI

from helper import (get_team_stats, 
                    get_player_stats,
                    position_dict,
                    get_type_by_id,
                    get_by_id,
                    team_stats,
                    player_stats,
                    get_user_prompt)

In [3]:
# OpenAI
openAI_Key = "YOUR_KEY"
client = OpenAI(api_key = openAI_Key)

In [4]:
MATCH_PROMPT = "The match for which you have to predict the final score is {team_1} vs. {team_2}."

SYSTEM_PROMPT = """\
You are a soccer expert with the latest knowledge about all European teams.
Your task is to predict the result of a 2024 European Championship game.

The user will provide you with the game information and details such as coach name, \
the team's performance during the qualification tournament, \
and statistics about all players of the respective team.

Before responding with the final score, you must analyze the provided information and justify the predicted score. 
Please take all information about the teams and players into account. 

Finally, respond in the following format:
Winner: TEAM_NAME
Final Score: TEAM_1: SCORE, TEAM_2: SCORE
"""

GAME_SUFFIX_PROMPT = "\n\n Now predict the outcome of the game following the instructions from above."

# Create prediction for individual matches

In [6]:
season_id = "22842"
schedules_new = get_by_id("schedules/seasons", season_id)
group_stage_schedules = schedules_new[0]

for round in group_stage_schedules["rounds"]:
    for fixture in round["fixtures"][:1]:
        
        # Construct match prompt
        team_1 = fixture['participants'][0]["name"]
        team_2 = fixture['participants'][1]["name"]
        game_prompt_final = MATCH_PROMPT.format(team_1=team_1, team_2=team_2)
        
        user_prompt = get_user_prompt(fixture,
                                      include_player_stats=True,
                                      prefix_prompt=game_prompt_final,
                                      suffix_prompt=GAME_SUFFIX_PROMPT)
        
        # Send request to OpenAI
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2
        )

        # Store prompts and predictions
        with open(f"{team_1}_{team_2}.json", "w") as f:
            json.dump({
                "system_prompt": SYSTEM_PROMPT,
                "user_prompt": user_prompt,
                "response": completion.choices[0].message.content
            }, f, indent=4)
        
        print(completion.choices[0].message.content)

### Analysis

#### Serbia
- **Coach**: Mladen Krstajić
- **Qualification Performance**:
  - Average goals scored per game: 1.88
  - Average goals conceded per game: 1.13
  - Ball possession: 60.13%
  - Clean sheets: 25%
  - Key Players: Aleksandar Mitrović (8.14 rating), Sergej Milinković-Savić (7.45 rating), Dušan Tadić (7.7 rating)
  - Strengths: High ball possession, consistent goal-scoring, strong attacking presence with Mitrović and Tadić.
  - Weaknesses: Conceding goals, especially between minutes 30 and 45.

#### England
- **Coach**: Gareth Southgate
- **Qualification Performance**:
  - Average goals scored per game: 2.75
  - Average goals conceded per game: 0.5
  - Ball possession: 62.5%
  - Clean sheets: 50%
  - Key Players: Bukayo Saka (7.83 rating), Harry Kane (7.36 rating), Kyle Walker (7.2 rating)
  - Strengths: Strong defense, high goal-scoring rate, effective in converting shots on target.
  - Weaknesses: Slightly lower ball possession compared to Serbia, but still high.

# Create predictions for group winner

In [8]:
season_id = "22842"
schedules_new = get_by_id("schedules/seasons", season_id)
group_stage_schedules = schedules_new[0]

group_name = "B" # Change this to get winners of other groups
groups = {
    "A": ["Germany", "Hungary", "Switzerland", "Scotland"],
    "B": ["Spain", "Croatia", "Italy", "Albania"],
    "C": ["Slovenia", "Serbia", "Denmark", "England"],
    "D": ["Poland", "Netherlands", "Austria", "France"],
    "E": ["Belgium", "Slovakia", "Romania", "Ukraine"],
    "F": ["Turkey", "Georgia", "Portugal", "Czech Republic"]
}
all_team_stats = {}
for round in group_stage_schedules["rounds"]:
    for fixture in round["fixtures"]:

        # Collect statistics for all teams
        team_prompt = get_user_prompt(fixture,
                                      include_player_stats=False,
                                      prefix_prompt="",
                                      suffix_prompt="",
                                      by_team=True)
        all_team_stats.update(team_prompt)

all_team_prompt = f"Here are the statistics of the performance of all teams from group {group_name} during the qualification:"

# Append stats of all teams of respective group
country_idx = 1
for country, stats in all_team_stats.items():
    if country in groups[group_name]:
        all_team_prompt += f"\n\n {country_idx}. {country}: {stats[2:]}"
        country_idx += 1

GROUP_WINNER_SYSTEM_PROMPT = f"""\
You are a soccer expert with the latest knowledge about all European teams.
Your task is to predict which team will win group {group_name} of the 2024 European Championship.

The user will provide you with the game information and details such as coach name and \
the team's performance during the qualification tournament.

Before responding the team name, you must analyze the provided statistics and justify why you chose the respective team.
Please take all information about the teams into account. 
"""

#Send request to OpenAI
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": GROUP_WINNER_SYSTEM_PROMPT},
        {"role": "user", "content": all_team_prompt}
    ],
    temperature=0.2
)

with open(f"winner.json", "w") as f:
    json.dump({
        "system_prompt": GROUP_WINNER_SYSTEM_PROMPT,
        "user_prompt": all_team_prompt,
        "response": completion.choices[0].message.content
    }, f, indent=4)

print(completion.choices[0].message.content)

Based on the provided statistics and performance during the qualification tournament, I predict that **Spain** will win Group B of the 2024 European Championship. Here’s a detailed analysis justifying this prediction:

### Spain
- **Performance and Consistency**: Spain won 87.5% of their qualification games, the highest win percentage among the teams in Group B. They also earned an average of 2.75 points per game, which is the highest in the group.
- **Offensive Strength**: Spain scored 25 goals, averaging 3.13 goals per game, which is significantly higher than the other teams. Their conversion rate of 15.65% and the fact that 37.41% of their shots were on target indicate a highly efficient attack.
- **Defensive Solidity**: Spain conceded only 5 goals in total, averaging 0.63 goals per game. They kept a clean sheet in 50% of their games, showing a strong defensive performance.
- **Ball Possession and Control**: With an average ball possession of 72.25%, Spain dominated their games, whi

# Create predictions for other statistics

In [9]:
season_id = "22842"
schedules_new = get_by_id("schedules/seasons", season_id)
group_stage_schedules = schedules_new[0]

all_team_stats = {}
for round in group_stage_schedules["rounds"]:
    for fixture in round["fixtures"]:

        # Collect statistics for all teams
        team_prompt = get_user_prompt(fixture,
                                      include_player_stats=False,
                                      prefix_prompt="",
                                      suffix_prompt="",
                                      by_team=True)
        all_team_stats.update(team_prompt)

all_team_prompt = "Here are the statistics of the performance of all participating teams during the qualification:"

# Append stats of all participating teams
for idx, (country, stats) in enumerate(all_team_stats.items(), start=1):
    all_team_prompt += f"\n\n {idx}. {country}: {stats[2:]}"

MOST_GOALS_SYSTEM_PROMPT = """\
You are a soccer expert with the latest knowledge about all European teams.
Your task is to predict which team of the 2024 European Championship will have the player with the most goals.

The user will provide you with the game information and details such as coach name and \
the team's performance during the qualification tournament.

Before responding the team name, you must analyze the provided statistics and justify why you chose the respective team.
Please take all information about the teams into account. 

Finally, respond in the following format:
Player with most goals: NAME
Team: TEAM_NAME
Estimated number of goals: ESTIMATE
"""

SEMI_FINAL_SYSTEM_PROMPT = """\
You are a soccer expert with the latest knowledge about all European teams.
Your task is to predict which 4 teams of the 2024 European Championship will make it to the semi finals.

The user will provide you with the game information and details such as coach name and \
the team's performance during the qualification tournament.

Before responding the team name, you must analyze the provided statistics and justify why you chose the respective team.
Please take all information about the teams into account. 

Finally, respond in the following format:
Team 1: TEAM_NAME
Team 2: TEAM_NAME
Team 3: TEAM_NAME
Team 4: TEAM_NAME
"""

WINNER_SYSTEM_PROMPT = """\
You are a soccer expert with the latest knowledge about all European teams.
Your task is to predict which team will win the 2024 European Championship.

The user will provide you with the game information and details such as coach name and \
the team's performance during the qualification tournament.

Before responding the team name, you must analyze the provided statistics and justify why you chose the respective team.
Please take all information about the teams into account. 
"""

#Send request to OpenAI
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": WINNER_SYSTEM_PROMPT},
        {"role": "user", "content": all_team_prompt}
    ],
    temperature=0.2
)

with open(f"winner.json", "w") as f:
    json.dump({
        "system_prompt": WINNER_SYSTEM_PROMPT,
        "user_prompt": all_team_prompt,
        "response": completion.choices[0].message.content
    }, f, indent=4)

print(completion.choices[0].message.content)

Based on the provided statistics, the team most likely to win the 2024 European Championship is **Portugal**. Here are the reasons for this prediction:

1. **Undefeated Record**: Portugal won all 10 of their qualification games, achieving a 100% win rate. This demonstrates their consistency and ability to perform under pressure.

2. **Defensive Strength**: Portugal conceded only 2 goals in total, averaging 0.2 goals per game. This indicates a very solid defense, which is crucial in tournament settings.

3. **Offensive Prowess**: Portugal scored 36 goals, averaging 3.6 goals per game. This is the highest among all the teams, showing their strong attacking capabilities.

4. **High Conversion Rate**: Portugal had a shot conversion rate of 20.98%, which is one of the highest among the teams. This efficiency in front of goal is a significant advantage.

5. **Ball Possession**: With an average ball possession of 67.7%, Portugal can control the game and dictate the pace, reducing the chances 